In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
os.chdir("../..")


In [19]:
import numpy as np
from enum import Enum
from numpy.typing import NDArray
from typing import Callable

from qiskit import QuantumCircuit
from qiskit.circuit.gate import Gate as QkGate
from qiskit.circuit.library import RXGate, RYGate, RZGate, CZGate
from qiskit.circuit import Parameter, Instruction


In [4]:
from qml.model.gate import GateInfo, get_gateset, Gate


In [7]:
Vector = list[float] | NDArray


In [28]:
class XRandomGenerator:

    SEED_RANGE = int(1e+8)

    def __init__(self, seed: int = None):
        if seed is None:
            seed = np.random.randint(0, self.SEED_RANGE)
        self.seed = seed
        self._rng = np.random.default_rng(seed)
        for key in dir(self._rng):
            if "__" in key:
                continue
            self.__setattr__(key, self._rng.__getattribute__(key))


In [20]:
def single_qubit_qargs(qc: QuantumCircuit, qubit_idx: int) -> list:
    return [qc.qubits[qubit_idx]]


def generate_double_qubit_args(spread: int = 1) -> Callable:
    def double_qubit_args(qc: QuantumCircuit, qubit_idx: int) -> list:
        return [
            qc.qubits[qubit_idx],
            qc.qubits[(qubit_idx+spread) % qc.num_qubits],
        ]
    return double_qubit_args


In [39]:
class GateInfo(Enum):

    def __init__(
            self,
            gate_class: QkGate,
            trainable: bool,
            multibit: bool,
            qargs: Callable,
    ):
        self.gate_class = gate_class
        self.trainable = trainable
        self.multibit = multibit
        self.qargs = qargs
    
    @classmethod
    def get(cls, obj):
        if isinstance(obj, Gateset):
            return obj
        if not isinstance(obj, str):
            raise ValueError(f"Gateset.get() requires Gateset or string, but {type(obj)} is given.")
        name = obj.upper()
        for g in cls:
            if name == g.name:
                return g
        raise ValueError(f"Incorrect obj is given; {obj}")


class Gateset:

    def __init__(self, num_qubits: int, seed: int = None):
        self._rng = XRandomGenerator(seed)
        self._nq = num_qubits
    
    @staticmethod
    def set_num_qubit(num_qubit: int) -> "Gateset":
        if num_qubit == 2:
            return Gateset2Qubit
        if num_qubit == 3:
            return Gateset3Qubit
        raise ValueError(f'num_qubit {num_qubit} is not supported.')
    
    @classmethod
    def get_at_random(cls, rng: XRandomGenerator = None):
        if rng is None:
            rng = np.random.default_rng()
        return rng.choice([g for g in cls])


class Gateset2Qubit(Gateset):

    RX = (RXGate, True, False, single_qubit_qargs)
    RY = (RYGate, True, False, single_qubit_qargs)
    RZ = (RZGate, True, False, single_qubit_qargs)
    CZ = (CZGate, False, True, generate_double_qubit_args(1))


class Gateset3Qubit(Gateset):

    RX = (RXGate, True, False, single_qubit_qargs)
    RY = (RYGate, True, False, single_qubit_qargs)
    RZ = (RZGate, True, False, single_qubit_qargs)
    CZ = (CZGate, False, True, generate_double_qubit_args(1))


gateset = Gateset.set_num_qubit(3)
gateset.get_at_random()



<Gateset3Qubit.RX: (<class 'qiskit.circuit.library.standard_gates.rx.RXGate'>, True, False, <function single_qubit_qargs at 0x117e09080>)>

In [ ]:
class Gate:

    def __init__(
            self,
            gate: Instruction,
            trainable: bool,
            multibit: bool,
            qubit: int,
            qargs_func: Callable,
    ):
        self._gate = gate
        self._trainable = trainable
        self._multibit = multibit
        self._qubit = qubit
        self._qargs_func = qargs_func

    def apply_to_qc(self, qc: QuantumCircuit):
        qargs = self._qargs_func(qc, self._qubit)
        qc.append(self._gate, qargs)

    @staticmethod
    def new_with_info(info: GateInfo, qubit: int, parameter: Parameter = None):
        gate = info.gate_class() if parameter is None else info.gate_class(parameter)
        return Gate(
            gate, info.trainable, info.multibit, qubit, info.qargs
        )

    @property
    def gate(self):
        return self._gate

    @property
    def trainable(self):
        return self._trainable

    @property
    def multi_qubit(self):
        return self._multi_qubit

    @property
    def qubit(self):
        return self._qubit


In [ ]:
class Unit:

    VALUE_MAX = 2 * np.pi

    def __init__(
            self,
            name: str,
            gates: list[Gate],
            params: list[Parameter],
            values: Vector
    ):
        self._name = name
        self._gates = gates
        self._params = params
        self._values = self.format(values)

    def feed_dict(self, values=None) -> dict[str, float]:
        if values is None:
            values = self._values
        if not hasattr(values, "__len__"):
            values = [values]
        
        assert len(values) == len(self.parameters), f"Length of values {len(values)} must be equal to number of parameters {len(self.num_param)}"
        values = self.format(values)
    
    @classmethod
    def format(cls, values: Vector) -> NDArray:
        values = np.asarray(values)
        if values.ndim == 0:
            values = values[np.newaxis]
        return values % cls.VALUE_MAX
    
    @property
    def parameters(self):
        return self.values
    
    @parameters.setter
    def parameters(self, values):
        assert len(values)
        self._values = self.format(values)
    
    @property
    def values(self):
        return self._values.copy()
    
    @property
    def gates(self):
        return [gate for gate in self._gates]
    
    @property
    def parameter_instances(self):
        return self._params
    
    @property
    def num_parameters(self):
        return len(self._params)


In [49]:
class UnitManager:

    def __init__(self, num_qubit: int, num_gate: int, gateset: int = None, seed: int = None):
        self.rng = XRandomGenerator(seed)
        self._nq = num_qubit
        self._ng = num_gate
        if gateset is None:
            gateset = Gateset.set_num_qubit(num_qubit)
        self.gateset = gateset
    
    def generate_random_unit(self, name: str = None):
        gates = [self.gateset.get_at_random() for _ in range(self._ng)]
        qubits = self.rng.integers(0, self._nq, size=self._ng)
        unit = self.generate_with_gateinfos_and_qubits(name, gates, qubits)
        print(unit)
        return 1
    
    def generate_with_gateinfos_and_qubits(
            self,
            name: str,
            gateinfos: list[Gateset],
            qubits: list[int],
    ):
        gates = []
        params = []

        for gateinfo, qubit in zip(gateinfos, qubits):
            if not gateinfo.trainable:
                gates.append(Gate.new_with_info(gateinfo, qubit))
                continue

            pname = f"param_{len(params)}"
            if name is not None:
                pname = name + "_" + pname
            param = Parameter(pname)
            params.append(param)
            gates.append(
                Gate.new_with_info(gateinfo, qubit, param)
            )
        
        values = np.zeros_like(params)

        return Unit(pname, gates, params)


uman = UnitManager(3, 3)
unit = uman.generate_random_unit()
print(unit)


AttributeError: 'Gateset3Qubit' object has no attribute 'multi_bit'

In [12]:
gateset = get_gateset(3)
name = "unit"
gates = [gateset]


1